In [1]:
import pandas as pd
import statistics
import numpy as np
import haversine as hs
from haversine import Unit
import math
from datetime import datetime
from scipy.stats import entropy
from scipy.stats import norm, kurtosis
import mysql.connector as msql
from mysql.connector import Error

In [2]:
def shannon_entropy(hist):
    side, count = np.unique(hist, return_counts=True)
    dist = count/len(hist)
    entropy_value = entropy(dist)
    return entropy_value
  
def kurtosis_fisher(dist):
    return kurtosis(dist)

def kurtosis_pearson(dist):
    return kurtosis(dist, fisher=False)

In [3]:
def get_features_by_traj(uids): 
    # unique trajectories
    count = 1
    total = len(uids)

    print('Processing trajectory')
    for uid in uids:
        print('---------------------------------------> (%s/%s)' % (count,total))
        
        uid = uid[0]
        print("------> Selecting from database")
        # filtering a single feature and trajectoy
        feature_query = "SELECT %s FROM trajectories.gpspoints WHERE identifier = '%s';" % ('latitude',uid)        
        mycursor.execute(feature_query)
        lats = mycursor.fetchall()

        feature_query = "SELECT %s FROM trajectories.gpspoints WHERE identifier = '%s';" % ('longitude',uid)        
        mycursor.execute(feature_query)
        lngs = mycursor.fetchall()
        
        feature_query = "SELECT %s FROM trajectories.gpspoints WHERE identifier = '%s';" % ('timestamp',uid)        
        mycursor.execute(feature_query)
        times = mycursor.fetchall()
        
        feature_query = "SELECT %s FROM trajectories.gpspoints WHERE identifier = '%s';" % ('transport_mode',uid)        
        mycursor.execute(feature_query)
        labels = mycursor.fetchall()
        
        # secondary values that will be used to calculate the features
        dists = []
        datetimes = []
        speeds = []
        accs = []
        
        print("------> Creating secondary features")
        if(len(lats)>=80):
            # for each trajectory, use primary values to calculate secondary
            for i in range(1,len(lats)):
                coord1 = (lats[i-1][0],lngs[i-1][0])
                coord2 = (lats[i][0],lngs[i][0])
                dists.append(hs.haversine(coord1,coord2,unit=Unit.METERS))

                start = times[i-1][0]
                end = times[i][0]
                datetimes.append(pd.Timedelta((end - start)).total_seconds())

                if datetimes[i-1] == 0:
                    speeds.append(0)
                    accs.append(0)
                else:
                    speeds.append(dists[i-1]/datetimes[i-1])
                    accs.append(speeds[i-1]/datetimes[i-1])

            # populate database
            row = (uid, max(dists), np.mean(dists), min(dists), statistics.variance(dists), shannon_entropy(dists), 
                   kurtosis_fisher(dists), kurtosis_pearson(dists),
                   max(speeds), np.mean(speeds), min(speeds), statistics.variance(speeds), shannon_entropy(speeds), 
                   kurtosis_fisher(speeds), kurtosis_pearson(speeds),
                   max(accs), np.mean(accs), min(accs), statistics.variance(accs), shannon_entropy(accs), 
                   kurtosis_fisher(accs), kurtosis_pearson(accs), labels[i][0])

            print("------> Inserting row into database")
            feature_query = '''INSERT INTO trajectories.features VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''        
            mycursor.execute(feature_query, row)
        
            count += 1

In [4]:
mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor(buffered=True)

try:
    if mydb.is_connected():

        uids_query = """SELECT DISTINCT identifier FROM trajectories.gpspoints;"""
        mycursor.execute(uids_query)
        uids = mycursor.fetchall()
        get_features_by_traj(uids)
        mydb.commit()

except Error as e:

    print("Error while connecting to gpspoints:", e)

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")


Processing trajectory
---------------------------------------> (1/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (3/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (4/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (5/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (6/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (7/3434)
----

------> Inserting row into database
---------------------------------------> (54/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (55/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (56/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (57/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (58/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (59/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
----------------------------------

---------------------------------------> (106/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (107/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (108/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (108/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (109/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (110/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (111/3434)
------> Selecting from database
------> Creating

------> Creating secondary features
------> Inserting row into database
---------------------------------------> (132/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (133/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (134/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (135/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (136/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (137/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into d

------> Creating secondary features
------> Inserting row into database
---------------------------------------> (180/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (181/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (182/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (183/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (184/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (185/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into d

...

------> Inserting row into database
---------------------------------------> (2822/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2823/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2824/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2825/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2826/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2827/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
----------------------

------> Inserting row into database
---------------------------------------> (2875/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2876/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2877/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2878/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2879/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2880/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
----------------------

---------------------------------------> (2928/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2929/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2930/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (2930/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2931/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2932/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2933/3434)
------> Selecting from database
------> C

------> Inserting row into database
---------------------------------------> (2980/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2981/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2982/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (2982/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2983/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2984/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2985/3434)
-----

---------------------------------------> (3001/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary features
---------------------------------------> (3002/3434)
------> Selecting from database
------> Creating secondary featu